In [12]:
import pandas as pd
import numpy as np
import findspark
findspark.init('C:\spark\spark-3.1.2-bin-hadoop3.2')
import pyspark

from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark import SparkConf
from pyspark.sql.functions import *
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

# Dataset 1

In [6]:
#Naive Bayes

weather = spark.read.option("inferschema","true").option("header","true").csv("weather.csv")
weather.show(5)

+--------+-----------+--------+-----+----+
| Outlook|Temperature|Humidity|Windy|Play|
+--------+-----------+--------+-----+----+
|   sunny|        hot|    high|false|  NO|
|   sunny|        hot|    high| true|  NO|
|overcast|        hot|    high|false| YES|
|   rainy|       mild|    high|false| YES|
|   rainy|       cool|  normal|false| YES|
+--------+-----------+--------+-----+----+
only showing top 5 rows



In [13]:
#P(yes)
total = weather.count()
total_yes = weather.where(col('Play')=='YES').count()




#P(no)
total_no = weather.where(col('Play')=='NO').count()


In [14]:
#outlook (col 1)
sunny = weather.where(col('Outlook')=='sunny')
s_c = sunny.count()
s_yes = sunny.where(col('Play')=='YES').count()
s_no = sunny.where(col('Play')=='NO').count()
sunny_yes = s_yes/total_yes
sunny_no = s_no/total_no
print(sunny_yes)
print(sunny_no)

overcast = weather.where(col('Outlook')=='overcast')
o_c = overcast.count()
o_yes = overcast.where(col('Play')=='YES').count()
o_no = overcast.where(col('Play')=='NO').count()
overcast_yes = o_yes/total_yes
overcast_no = o_no/total_no
print(overcast_yes)
print(overcast_no)

rainy = weather.where(col('Outlook')=='rainy')
r_yes = rainy.where(col('Play')=='YES').count()
r_no = rainy.where(col('Play')=='NO').count()
rainy_yes = r_yes/total_yes
rainy_no = r_no/total_no
print(rainy_yes)
print(rainy_no)

0.2222222222222222
0.6
0.4444444444444444
0.0
0.3333333333333333
0.4


In [15]:
#Temprature(col 2)
cool = weather.where(col('Temperature')=='cool')
c_yes = cool.where(col('Play')=='YES').count()
c_no = cool.where(col('Play')=='NO').count()
cool_yes = c_yes/total_yes
cool_no = c_no/total_no
print(cool_yes)
print(cool_no)

hot = weather.where(col('Temperature')=='hot')
h_yes = hot.where(col('Play')=='YES').count()
h_no = hot.where(col('Play')=='NO').count()
hot_yes = h_yes/total_yes
hot_no = h_no/total_no
print(hot_yes)
print(hot_no)

mild = weather.where(col('Temperature')=='mild')
m_yes = mild.where(col('Play')=='YES').count()
m_no = mild.where(col('Play')=='NO').count()
mild_yes = m_yes/total_yes
mild_no = m_no/total_no
print(mild_yes)
print(mild_no)

0.3333333333333333
0.2
0.2222222222222222
0.4
0.4444444444444444
0.4


In [16]:
#windy(col 3)
windy = weather.where(col('Windy')=='false')
w_yes = windy.where(col('Play')=='YES').count()
w_no = windy.where(col('Play')=='NO').count()
windy_yes = w_yes/total_yes
windy_no = w_no/total_no
print(windy_yes)
print(windy_no)

windyy = weather.where(col('Windy')=='true')
ww_yes = windyy.where(col('Play')=='YES').count()
ww_no = windyy.where(col('Play')=='NO').count()
windyy_yes = ww_yes/total_yes
windyy_no = ww_no/total_no
print(windyy_yes)
print(windyy_no)

0.6666666666666666
0.4
0.3333333333333333
0.6


In [17]:
#Humidity(col 4)
high = weather.where(col('Humidity')=='high')
hu_yes = high.where(col('Play')=='YES').count()
hu_no = high.where(col('Play')=='NO').count()
high_yes = hu_yes/total_yes
high_no = hu_no/total_no
print(high_yes)
print(high_no)

normal = weather.where(col('Humidity')=='normal')
n_yes = normal.where(col('Play')=='YES').count()
n_no = normal.where(col('Play')=='NO').count()
normal_yes = n_yes/total_yes
normal_no = n_no/total_no
print(normal_yes)
print(normal_no)

0.3333333333333333
0.8
0.6666666666666666
0.2


In [22]:
#P(YES | rainy, hot, normal, not windy) = p(yes)*p(rainy|yes)*p(hot|yes)*p(normal|yes)*p(not windy|yes)
p_yes_rhnw = total_yes * rainy_yes * normal_yes * hot_yes * windy_yes
print(p_yes_rhnw)

0.2962962962962963


In [23]:
#P(No | rainy, hot, normal, not windy) = p(no)*p(rainy|no)*p(hot|no)*p(normal|no)*p(not windy|no)
p_no_rhnw = total_no * rainy_no * normal_no * hot_no * windy_no
print(p_no_rhnw)

0.06400000000000002


# Dataset 2

In [13]:
salary = spark.read.option("inferschema","true").option("header","true").csv("gnb.csv")
salary.show(5)

+---+--------+---------+----------+-------+
| ID|    Type|   Amount|OldBalance|isFraud|
+---+--------+---------+----------+-------+
|  1| PAYMENT|  9839.64|  170136.0|      0|
|  2| PAYMENT|  1864.28|   21249.0|      0|
|  3|TRANSFER|    181.0|     181.0|      1|
|  4|CASH_OUT|    181.0|     181.0|      1|
|  5|TRANSFER|865948.49| 865948.49|      1|
+---+--------+---------+----------+-------+
only showing top 5 rows



In [14]:
total = salary.count()
total_no = salary.where(col('isFraud')==0)
total_yes = salary.where(col('isFraud')==1)

In [15]:
total_yes.show()

+---+--------+---------+----------+-------+
| ID|    Type|   Amount|OldBalance|isFraud|
+---+--------+---------+----------+-------+
|  3|TRANSFER|    181.0|     181.0|      1|
|  4|CASH_OUT|    181.0|     181.0|      1|
|  5|TRANSFER|865948.49| 865948.49|      1|
|  6|CASH_OUT| 86120.87|   94120.4|      1|
| 10| PAYMENT|   4300.0|   4305.75|      1|
+---+--------+---------+----------+-------+



In [16]:
total_no.show()

+---+--------+---------+----------+-------+
| ID|    Type|   Amount|OldBalance|isFraud|
+---+--------+---------+----------+-------+
|  1| PAYMENT|  9839.64|  170136.0|      0|
|  2| PAYMENT|  1864.28|   21249.0|      0|
|  7|CASH_OUT|  5378.12|   39005.0|      0|
|  8| PAYMENT|  6027.23| 175690.81|      0|
|  9|TRANSFER|539880.48|   56395.0|      0|
+---+--------+---------+----------+-------+



In [32]:
total_no.select(stddev("Salary")).show()

+-------------------+
|stddev_samp(Salary)|
+-------------------+
| 36295.239969511764|
+-------------------+



In [33]:
total_yes.select(stddev("Salary")).show()

+-------------------+
|stddev_samp(Salary)|
+-------------------+
|  29373.61952176687|
+-------------------+



In [17]:
total_no.select(mean("Amount")).show()

+-----------+
|avg(Amount)|
+-----------+
|  112597.95|
+-----------+



In [18]:
total_yes.select(mean("Amount")).show()

+-----------+
|avg(Amount)|
+-----------+
| 191346.272|
+-----------+



In [19]:
total_yes.select(mean("OldBalance")).show()

+---------------+
|avg(OldBalance)|
+---------------+
|     192947.328|
+---------------+



In [20]:
total_no.select(mean("OldBalance")).show()

+---------------+
|avg(OldBalance)|
+---------------+
|      92495.162|
+---------------+



In [37]:
total_yes.select(stddev("Age")).show()

+----------------+
|stddev_samp(Age)|
+----------------+
|4.68533680244878|
+----------------+



In [23]:
total_no.count()/salary.count()

0.5

In [ ]:
total_no.select(stddev("Age")).show()

In [38]:
total_no.select(mean("Age")).show()

+--------+
|avg(Age)|
+--------+
|    45.5|
+--------+



In [30]:
a = total_no.filter((col("Type")=="TRANSFER"))
a.select(mean("Amount")).show()

+-----------+
|avg(Amount)|
+-----------+
|  539880.48|
+-----------+



# Dataset 3

In [41]:
bayes = spark.read.option("inferschema","true").option("header","true").csv("bayes.csv")
bayes.show(5)

+------+------+--------+------+
| color|  type|  origin|stolen|
+------+------+--------+------+
|   red|sports|domestic|   yes|
|   red|sports|domestic|    no|
|   red|sports|domestic|   yes|
|yellow|sports|domestic|    no|
|yellow|sports|imported|   yes|
+------+------+--------+------+
only showing top 5 rows



In [44]:
#Total
total_yes = (bayes.filter(col('stolen')=='yes').count())/(bayes.count())
total_no = (bayes.filter(col('stolen')=='no').count())/(bayes.count())
print(total_yes)
print(total_no)

0.5
0.5


In [46]:
#color = red and yellow
red = bayes.filter(col('color')=='red')
red_yes = (red.filter(col('stolen')=='yes').count())/(red.count())
red_no = (red.filter(col('stolen')=='no').count())/(red.count())
print(red_yes)
print(red_no)

yellow = bayes.filter(col('color')=='yellow')
yellow_yes = (yellow.filter(col('stolen')=='yes').count())/(yellow.count())
yellow_no = (yellow.filter(col('stolen')=='no').count())/(yellow.count())
print(yellow_yes)
print(yellow_no)

0.6
0.4
0.4
0.6


In [47]:
#type - sports and suv

In [48]:
#origin - domestic and imported

In [ ]:
#mean var stddev of each colummn(yes and no)

# Pandas

In [1]:
import pandas as pd
df = pd.read_csv("weather.csv")
df

,Outlook,Temperature,Humidity,Windy,Play
0,sunny,hot,high,False,NO
1,sunny,hot,high,True,NO
2,overcast,hot,high,False,YES
3,rainy,mild,high,False,YES
4,rainy,cool,normal,False,YES
5,rainy,cool,normal,True,NO
6,overcast,cool,normal,True,YES
7,sunny,mild,high,False,NO
8,sunny,cool,normal,False,YES
9,rainy,mild,normal,False,YES


In [11]:
pivot = pd.pivot_table(data = df,index='Play',aggfunc='count')
pivot

,Humidity,Outlook,Temperature,Windy
Play,,,,
NO,5,5,5,5
YES,9,9,9,9
